# Step10, 테스트

## 10.1 파이썬 단위 테스트

파이썬으로 테스트할 때는 표준 라이브러리에 포함된 unittest를 사용하면 편하다.

In [8]:
import unittest 
from DeZero import *

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)      # y값과 예측값이 같은지 


assertEqual(y.data, expected)는 두 객체가 서로 동일한지 여부를 판정  
이것 뿐만아니라 unittest에는 assertGreater와 assertTrue등이 있다.

## 10.2 square 함수의 역전파 테스트 


In [11]:
import unittest 
from DeZero import *

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)      # y값과 예측값이 같은지 

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

## 10.3 기울기 확인을 이용한 자동 테스트

위 셀에서는 미분의 기댓값을 손으로 계산해 입력하였다. 이 부분을 자동화한다.

**기울기 확인** : 수치 미분으로 구한 결과와 역전파로 구한 결과를 비교하여 그 차이가 크면 역전파 구현에 문제가 있다고 판단하는 검증 기법

기울기 확인은 (기대값을 몰라도) 입력값만 준비하면 되므로 테스트 효율을 높혀준다.

In [14]:
# 중앙 차분, 수치미분
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1))         # 무작위 입력값을 하나 생성
        y = square(x)
        y.backward()                            # 역전파로 미분값
        num_grad = numerical_diff(square, x)    # 수치미분으로 미분값
        flg = np.allclose(x.grad, num_grad)     # 두 방법의 미분값을 비교 가까우면 True
        self.assertTrue(flg)

np.allclose(a, b)는 ndarray 인스턴스인 a와 b의 값이 가까운지 판정  

얼마나 가까워야 가까운것인지 기준은 np.allclose(a, b, rtol=1e-05, atol=1e-08)과 같이 인수 rtol과 atol로 지정  
조건을 만족하면 True를 반환 

|a - b| <= (atol + rtol * |b|)

## 10.4 테스트 정리 

tests 디렉토리에 모야져 있음

테스트 파일들을 한번에 실행하여 확인하고 싶으면 

python -m unittest discover tests

tests는 파일들이 들어있는 디렉토리 